In [1]:
import pandas as pd
import numpy as np
import sys
import math
from PIL import Image
from PIL.ExifTags import TAGS, GPSTAGS
from math import cos, asin, sqrt, pi

In [2]:
def get_exif(filename):
    return Image.open(filename)._getexif()

In [3]:
def get_labeled_exif(exif):
    labeled = {}
    for (key, val) in exif.items():
        labeled[TAGS.get(key)] = val

    return labeled

In [4]:
def get_geotagging(exif):
    if not exif:
        raise ValueError("No EXIF metadata found")

    geotagging = {}
    for (idx, tag) in TAGS.items():
        if tag == 'GPSInfo':
            if idx not in exif:
                raise ValueError("No EXIF geotagging found")

            for (key, val) in GPSTAGS.items():
                if key in exif[idx]:
                    geotagging[val] = exif[idx][key]

    return geotagging

In [5]:
def get_decimal_from_dms(dms, ref):

    # https://stackoverflow.com/questions/64405326/django-exif-data-ifdrational-object-is-not-subscriptable
    degrees = dms[0]
    minutes = dms[1] / 60.0
    seconds = dms[2] / 3600.0

    if ref in ['S', 'W']:
        degrees = -degrees
        minutes = -minutes
        seconds = -seconds

    return round(degrees + minutes + seconds, 5)

In [6]:
def get_coordinates(geotags):
    lat = get_decimal_from_dms(geotags['GPSLatitude'], geotags['GPSLatitudeRef'])

    lon = get_decimal_from_dms(geotags['GPSLongitude'], geotags['GPSLongitudeRef'])

    return (lat,lon)

In [7]:
def cal_dis(amen, lat2, lon2):
    
    lat1 = amen['lat']
    lon1 = amen['lon']
    
    p = pi / 180
    a = 0.5-np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    b = 12742*np.arcsin(np.sqrt(a))*1000
   
    return b

In [8]:
def cal_dis2(lat1,lon1, lat2, lon2):
    
    p = pi / 180
    a = 0.5-np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    b = 12742*np.arcsin(np.sqrt(a))*1000
   
    return b

In [9]:
def Create_Path(from_lat,from_lon,to_lat,to_lon,amen, Qudrant):
    if Qudrant == 'I':
        amen = amen[from_lat<=amen['lat']]
        amen = amen[amen['lat']<=to_lat]
        amen = amen[from_lon<=amen['lon']]
        amen = amen[amen['lon']<=to_lon]
    elif Qudrant == 'II':
        amen = amen[from_lat<=amen['lat']]
        amen = amen[amen['lat']<=to_lat]
        amen = amen[from_lon>=amen['lon']]
        amen = amen[amen['lon']>=to_lon]
    elif Qudrant == 'III':
        amen = amen[from_lat>=amen['lat']]
        amen = amen[amen['lat']>=to_lat]
        amen = amen[from_lon>=amen['lon']]
        amen = amen[amen['lon']>=to_lon]
    else:    
        amen = amen[from_lat>=amen['lat']]
        amen = amen[amen['lat']>=to_lat]
        amen = amen[from_lon<=amen['lon']]
        amen = amen[amen['lon']<=to_lon]
    
    path_df = amen[0:0]
    
    amen['distance'] = amen.apply(cal_dis, lat2 = from_lat, lon2 = from_lon, axis=1)
    amen = amen.sort_values(by=['distance'])
    
    next_place = amen.head(1)
    next_plcae = next_place.reset_index(drop=True, inplace=True)
    path_df = path_df.append(next_place,ignore_index=True)
    
    if amen.shape[0]==1:
        return path_df
    
    next_amen = amen.iloc[1:,:]
    

    if next_place['amenity'][0]=='bar':
        next_amen = next_amen[next_amen['amenity'].isin(['bar'])!= True]
    elif next_place['amenity'][0]=='restaurant':
        next_amen = next_amen[next_amen['amenity'].isin(['restaurant'])!= True]
    elif next_place['amenity'][0]=='cafe':
        next_amen = next_amen[next_amen['amenity'].isin(['cafe'])!= True]
    
    next_amen.drop(['distance'],axis=1)
    
    continue_path = Create_Path(next_place['lat'].values[0], next_place['lon'].values[0],to_lat,to_lon,next_amen, Qudrant)
    path_df = path_df.append(continue_path, ignore_index = True)
    
    return path_df
    
    #print(next_amen)

In [10]:
def Create_directory(from_direct,to_direct,amen):
    from_lat = from_direct['lat'][0]
    from_lon = from_direct['lon'][0]
    to_lat = to_direct['lat'][0]
    to_lon = to_direct['lon'][0]
    if from_lat <= to_lat :
        if from_lon <= to_lon:
            Qudrant = 'I'
        else:
            Qudrant = 'II'
    else:
        if from_lon <= to_lon:
            Qudrant = 'IV'
        else:
            Qudrant = 'III'
        
    
    path = Create_Path(from_lat,from_lon,to_lat,to_lon,amen, Qudrant)
    path = path.drop(['distance'],axis=1)
    return from_direct.append(path, ignore_index=True)
    

In [11]:
def main (start_hotel_id, B_image, C_image):
    
    #image = 'p1.jpg'
    exif = get_exif(B_image)
    labeled = get_labeled_exif(exif)
    geotags = get_geotagging(exif)
    Blat, Blon = get_coordinates(geotags)
    B_place_lat = float(Blat)
    B_place_lon = float(Blon)
    
    exif = get_exif(C_image)
    labeled = get_labeled_exif(exif)
    geotags = get_geotagging(exif)
    Clat, Clon = get_coordinates(geotags)
    C_place_lat = float(Clat)
    C_place_lon = float(Clon)
    
    
    hotel = pd.read_csv("listings.csv")
    selected_hotel = hotel[hotel['id']==start_hotel_id].reset_index(drop=True)

    selected_hotel_lat = selected_hotel['latitude'][0]
    selected_hotel_lon = selected_hotel['longitude'][0]
    selected_hotel_name = selected_hotel['name'][0]
    
    
    
    #Travel mode
    total_distance = cal_dis2(selected_hotel_lat, selected_hotel_lon, B_place_lat, B_place_lon) + cal_dis2(B_place_lat, B_place_lon, C_place_lat, C_place_lon)
    if total_distance < 1500:
        print('The total distance from hotel to the destination less than 1500m. We sugguest the travel mode is walking')
    elif total_distance >= 1500 and total_distance < 5000:
        print('The total distance from hotel to the destination more than 1500m and less than 5000m. We sugguest the travel mode is biking')
    else:
        print('The total distance from hotel to the destination more than 5000m. We sugguest the travel mode is driving')
    
    

    amen = pd.read_json('amenities-vancouver.json.gz', lines=True)
    addition_amen = pd.read_csv('other_amenities.csv')
    amen = amen.drop(['timestamp','tags'], axis=1)
    amen = amen.dropna().reset_index(drop= True)
    df = [amen, addition_amen]
    amen = pd.concat(df).reset_index(drop = True)

    hotel_amenity = amen[0:0]
    hotel_amenity.loc[0,'lat'] = selected_hotel_lat
    hotel_amenity.loc[0,'lon'] = selected_hotel_lon
    hotel_amenity.loc[0,'amenity'] = 'hotel'
    hotel_amenity.loc[0,'name'] = selected_hotel_name


    filt = ['arts_centre', 'restaurant','bar','cafe','casino','gambling', 'cinema', 'clock','museum','park','university','beach','lake', 'shopping_centre','conference_centre','theatre']
    amen = amen[amen['amenity'].isin(filt) == True]

    #selected_amen_B = amen[amen['amenity']=='arts_centre'].reset_index(drop=True)
    #selected_amen_B_lat = selected_amen_B['lat'][0]
    #selected_amen_B_lon = selected_amen_B['lon'][0]
    B_place = amen[0:0]
    B_place.loc[0,'lat'] = B_place_lat
    B_place.loc[0,'lon'] = B_place_lon
    B_place.loc[0,'amenity'] = 'B_place'
    B_place.loc[0,'name'] = 'B_place'
    amen = amen.append(B_place)
    

    AtoB_Path = Create_directory(hotel_amenity,B_place,amen)
    AtoB_Path.to_csv('Path_from_hotel_to_first')

    second_amen = amen[amen['name'].isin(AtoB_Path['name'])!=True]
    C_place = amen[0:0]
    C_place.loc[0,'lat'] = C_place_lat
    C_place.loc[0,'lon'] = C_place_lon
    C_place.loc[0,'amenity'] = 'C_place'
    C_place.loc[0,'name'] = 'C_place'
    second_amen = second_amen.append(C_place)

    BtoC_Path = Create_directory(B_place,C_place,second_amen)
    BtoC_Path.to_csv('Path_from_first_to_second')

    third_amen = amen[amen['name'].isin(BtoC_Path['name'])!=True]
    third_amen = third_amen.append(hotel_amenity)
    
    CtoA_Path = Create_directory(C_place,hotel_amenity, third_amen)
    CtoA_Path.to_csv('Path_from_second_to_hotel')


In [12]:
if __name__=='__main__':
    #Hotel_ID = sys.argv[1]
    Hotel_ID = 26047
    
    #B_image = sys.argv[2]
    B_image = 'B.jpg'
    
    #C_image = sys.argv[3]
    C_image = 'C.jpg'
    main(Hotel_ID, B_image, C_image)

The total distance from hotel to the destination more than 1500m and less than 5000m. We sugguest the travel mode is biking


<ipython-input-11-d87dc46378cc>:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_amenity.loc[0,'lat'] = selected_hotel_lat
<ipython-input-11-d87dc46378cc>:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  B_place.loc[0,'lat'] = B_place_lat
<ipython-input-11-d87dc46378cc>:72: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  C_place.loc[0,'lat'] = C_place_lat
